## Data Cleaning



In [ ]:
#importing necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('/content/drive/MyDrive/Project_files/Data_Science_Internship - Dump.csv')
df=df.drop("Unnamed: 0",axis=1)

In [ ]:
filter=df[(df['status'] != 'LOST') & (df['status']!='WON')].index
df.drop(filter , inplace=True)

for feat in df.columns:
    df[feat].fillna(df[feat].mode()[0], inplace=True)
df=df.drop(columns='movein',axis=1)

#Checking the missing values of dataframe
df.isnull().sum()

Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
room_type         0
lead_id           0
dtype: int64

In [ ]:
#Storing the numerical features of the dataframe

categorical_features=[features for features in df.columns if df[features].dtypes=='O']

le=LabelEncoder()
for feat in categorical_features:
    df[feat] = le.fit_transform(df[feat].astype(str))

## Feature Selection

In [ ]:
# CALCULATING THE ENTROPY OF ALL THE NON-TARGET COLUMNS

from scipy.stats import entropy

df2=df.copy()
df2 = df2.iloc[:,:]
slice_data = df2.iloc[:, 0:df2.shape[1]]
entrpy = []

for k in range(slice_data.shape[1]):
  colom= slice_data[slice_data.columns[k]].tolist()
  colom=np.array(colom)
  colom = colom.astype('float32')
  e=entropy(colom, base=2)
  entrpy.append(e)

entrpy=np.array(entrpy)

#sorting the columns according to entropy

sort_index=np.argsort(entrpy)[::-1]
sort_index=list(sort_index)

sort_index.remove(4)


In [ ]:
fx=[]
for i in range(10):
  fx.append(df.columns[sort_index[i]])
print("Top 10-Features Selected for model training ",fx)

Top 10-Features Selected for model training  ['des_country', 'source', 'utm_source', 'lost_reason', 'source_country', 'source_city', 'budget', 'des_city', 'lead_id', 'Agent_id']


## Model Training

In [ ]:
X=np.array(df[fx])
y=df['status']

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=104,train_size=0.8, shuffle=True)


from sklearn.ensemble import RandomForestClassifier

# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100)

# Training the model on the training dataset

# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)



RandomForestClassifier()

## Testing

In [ ]:
# performing predictions on the test dataset
y_pred = clf.predict(X_test)

# metrics are used to find accuracy or error
from sklearn import metrics

# using metrics module for Model Evaluation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, y_pred))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, y_pred))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.9826208981001727
PRECISION OF THE MODEL:  0.8589743589743589
RECALL OF THE MODEL:  0.8801313628899836
F1 SCORE OF THE MODEL:  0.8694241686942418


## Lead Score of X_test

In [ ]:
lead_score = clf.predict_proba(X_test)
lead_score=pd.DataFrame(lead_score)
print("Lead Score of top 50 X_test:",np.array(lead_score.head(50)[1]*100))

Lead Score of top 50 X_test: [23.  0. 65.  0.  0.  0.  0.  0.  0.  1. 24.  0.  1.  2.  0.  0.  0.  0.
  0.  0.  2.  0. 34. 80.  0.  0.  0.  0.  0.  0.  0.  0. 80.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  2.  0.  2. 81.  0.  0.]
